# Imports

In [1]:
import os
import sys
sys.path.append('..')

import torch

from src.models.yolo.yolo import load_yolov8_detector
from src.models.sam.sam import load_sam_predictor, segment_images_from_folder

# Data

In [2]:
DATA_DIR = r""

In [3]:
def filter_hidden_folders(folder):
    if folder[0] == '.':
        return False
    return True

In [ ]:
folders_list = os.listdir(DATA_DIR)
folders_list = list(filter(filter_hidden_folders, folders_list))
folders_list

In [5]:
def count_images(image_folder_path):
    images_num = len(os.listdir(image_folder_path))
    return images_num

def summarize_folders(data_dir, folders_list):
    folders_info = {}
    images_sum = 0
    
    for folder in folders_list:
        images_num = count_images(os.path.join(data_dir, folder))
        folders_info[folder] = images_num
        images_sum += images_num
        
    print(f"There is {images_sum} images:")
    for folder in folders_info.keys():
        print(f"\t{folder} folder has {folders_info[folder]} images")
    

In [ ]:
summarize_folders(DATA_DIR, folders_list)

# SAM+YOLO Labeling

Yolo detector:

In [7]:
custom_yolo_checkpoint_path = r""
detector = load_yolov8_detector(custom_yolo_checkpoint_path)

SAM:

In [8]:
sam_checkpoint = r""
model_type = "vit_h"
device = "cuda" if torch.cuda.is_available() else "cpu"

predictor = load_sam_predictor(checkpoint_path=sam_checkpoint, model_type=model_type, device=device)

## Labeling

In [ ]:
folder = folders_list[3]
folder

In [11]:
target_length=1024
narrowing = 0.20
erode_iterations = 1

In [13]:
source_dir = os.path.join(DATA_DIR, folder)
output_dir = os.path.join(DATA_DIR, folder + "_masks")

processes_num = 2

In [ ]:
segment_images_from_folder(
    source_dir,
    output_dir,
    detector, predictor,
    target_length=target_length,
    narrowing=narrowing,
    erode_iterations=erode_iterations,
    processes_num=processes_num
)